In [0]:
%run ./0.Config

In [0]:
dbutils.fs.rm("abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/Trackers/clarification_email_tracker",True)

In [0]:
def send_outlier_price_clarifications(_: str) -> str:
    """
    Detects outlier quotes from vendor submissions and sends a clarification email to each vendor (only once per route/truck combo).
    Tracks emails sent and saves a detailed Delta table in ADLS.
    """
    try:
        yag = yagmail.SMTP(user=EMAIL_USER, password=EMAIL_PASSWORD)

        # 1) Load and preprocess vendor responses
        input_path = (
            "abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/"
            "vendor_response/Vendor_Response.csv")
        df = (
            spark.read.option("header","true")
                 .option("inferSchema","true")
                 .csv(input_path)
            .toPandas()
        )
        # type conversions
        df['Price per Truck'] = pd.to_numeric(df['Price per Truck'], errors='coerce')
        df['Count'] = pd.to_numeric(df['Count'], errors='coerce')
        df['Total Cost'] = pd.to_numeric(df['Total Cost'], errors='coerce')
        df['Submitted At'] = pd.to_datetime(df['Submitted At'], errors='coerce')
        df = df.dropna(subset=[
            "Vendor Name","Vendor Email","Region",
            "Route ID","Truck Type","Price per Truck","Count"
        ])

        # 2) Flag outliers per truck type
        def flag_outliers(group):
            q1, q3 = group['Price per Truck'].quantile([0.25,0.75])
            iqr = q3 - q1
            lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
            group['lower_bound'], group['upper_bound'] = lower, upper
            group['outlier_type'] = None
            if iqr>0 and group['Price per Truck'].nunique()>1:
                group.loc[group['Price per Truck']<=lower,'outlier_type']='low'
                group.loc[group['Price per Truck']>=upper,'outlier_type']='high'
            return group

        df_flagged = (
            df.groupby('Truck Type',group_keys=False)
              .apply(flag_outliers)
              .reset_index(drop=True))
        outliers = df_flagged[df_flagged['outlier_type'].notnull()].copy()

        # 3) Setup Delta tracker
        tracker_path = (
            "abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/Trackers/clarification_email_tracker"
        )
        tracker_schema = StructType([
            StructField("Vendor_Name", StringType(), True),
            StructField("Vendor_Email", StringType(), True),
            StructField("Route_ID", StringType(), True),
            StructField("Truck_Type", StringType(), True),
            StructField("quoted_Price_per_Truck", DoubleType(), True),
            StructField("Submitted_At", TimestampType(), True),
            StructField("email_sent_at", TimestampType(), True),
            StructField("outlier_type", StringType(), True),
        ])
        try:
            tracker_spark = spark.read.format("delta").load(tracker_path)
        except AnalysisException:
            # create empty Delta with full schema
            empty = spark.createDataFrame([], tracker_schema)
            empty.write.format("delta").mode("overwrite").save(tracker_path)
            tracker_spark = spark.read.format("delta").load(tracker_path)
        tracker_df = tracker_spark.toPandas()
        already_sent = set(
            (r['Vendor_Email'],r['Route_ID'],r['Truck_Type'])
            for r in tracker_df.to_dict('records'))

        # 4) Early exit if nothing new
        needed = set(
            tuple(row) for row in outliers[[
                'Vendor Email','Route ID','Truck Type'
            ]].values)
        if needed.issubset(already_sent):
            return "Final Answer: ℹ️ No new outlier quotes. All vendors notified already."

        # 5) Prepare LLM prompt
        model = ChatDatabricks(endpoint="databricks-llama-4-maverick")
        prompt = PromptTemplate(
            input_variables=["Vendor Name","clarification_details"],
            template=(
"""You are a procurement officer at Coca Cola Pvt Ltd reviewing vendor quotations received in response to an RFQ.

Vendor: {Vendor Name}

The following quotes you submitted appear to have unusually high or low Price per Trucks per truck:
{clarification_details}

Please review and clarify these pricing discrepancies. If there was an input error, kindly resubmit corrected quotations using the following link within 2 days:
https://genai-automation-5fciora2kjgdcowb3v9ymk.streamlit.app/

Keep the tone professional and courteous. Do **not** assume any correct values.

End the email with the following sign-off:

Best regards,  
Coca Cola Pvt Ltd  
procurement@cocacola.com

Return only the email body"""
            ))
        llm_chain = prompt | model

        # 6) Send emails and collect new tracker rows
        new_records = []
        sent_count = 0
        for email, grp in outliers.groupby('Vendor Email'):
            # filter out already sent
            to_send = grp[
                ~grp.apply(
                    lambda x: (x['Vendor Email'],x['Route ID'],x['Truck Type']) in already_sent,
                    axis=1
                )]
            if to_send.empty:
                continue

            # craft details
            lines = [
                f"- Route: {r['Route ID']} | Truck: {r['Truck Type']} | "
                f"Price: ₹{r['Price per Truck']} ({r['outlier_type']}) "
                for _,r in to_send.iterrows()]
            
            body = llm_chain.invoke({
                "Vendor Name": to_send.iloc[0]['Vendor Name'],
                "clarification_details": "\n".join(lines)
            })
            cont = getattr(body,'content',body)
            yag.send(to=email,subject="Clarification Requested",contents=cont)
            sent_count += 1

            # track each row in detail
            for _,r in to_send.iterrows():
                new_records.append({
                    "Vendor_Name": r['Vendor Name'],
                    "Vendor_Email": r['Vendor Email'],
                    "Route_ID": r['Route ID'],
                    "Truck_Type": r['Truck Type'],
                    "quoted_Price_per_Truck": r['Price per Truck'],
                    "Submitted_At": r['Submitted At'],
                    "email_sent_at": datetime.now(),
                    "outlier_type": r['outlier_type'],
                })

        # 7) Append new entries to Delta
        if new_records:
            new_df = spark.createDataFrame(pd.DataFrame(new_records), tracker_schema)
            new_df.write.format("delta").mode("append").save(tracker_path)

        return f"Final Answer: 📨 Sent {sent_count} clarification emails. Tracker Delta at {tracker_path}"
    except Exception as e:
        return f"❌ Error occurred: {e}"

# Wrap as single-input Tool
outlier_tool = Tool(
    name="send_outlier_price_clarifications",
    func=send_outlier_price_clarifications,
    description="Detects outlier quotes and emails vendors once each",
    return_direct=True
)

In [0]:
tools = [outlier_tool]

agent = initialize_agent(
    tools=tools,
    llm=ChatDatabricks(endpoint="databricks-llama-4-maverick"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Requesting clarification on outlier quotes from various vendors as per the latest RFQ.")